<a href="https://colab.research.google.com/github/vidhi-sys/GenAI_LangChain_and_RAG/blob/main/Fine_tuning_using_a_pretrain_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#!pip install datasets

FineTuning IMDB

In [10]:
from datasets import load_dataset
data=load_dataset("imdb")

In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [12]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
text="i couldnt find anything good in the movie it was a waste of money "
tokenized_text=tokenizer(text)
print(tokenized_text)


{'input_ids': [101, 1045, 2481, 2102, 2424, 2505, 2204, 1999, 1996, 3185, 2009, 2001, 1037, 5949, 1997, 2769, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [13]:
input_id=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
print(input_id)

[1045, 2481, 2102, 2424, 2505, 2204, 1999, 1996, 3185, 2009, 2001, 1037, 5949, 1997, 2769]


In [14]:
def tokenize_fun(batch):
  return tokenizer(batch["text"],padding="max_length",truncation=True)

In [15]:
tokenized_data=data.map(tokenize_fun,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification
import torch
training_args=TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.0,
    report_to="none")

In [17]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"]
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
trainer.train()

Step,Training Loss
500,0.393000
1000,0.282600
1500,0.232200
2000,0.165700


KeyboardInterrupt: 

In [ ]:
results=trainer.evaluate()
print(results)

Step,Training Loss
500,0.393000
1000,0.282600
1500,0.232200
2000,0.165700
